# Wind Analysis

This notebook outlines a good example of getting started for the wind analysis. It demonstrates a couple of new, powerful tools and an example of the processing and plotting.

First, import the packages that we will be using. One new package we will be using is xarray. It is designed for working with oceanographic and other earth science datasets, particularly
netCDF, HDF5 (common in satellite/NASA products), and other multi-dimensional data.
It will also introduce matplotlib, a basic plotting library that produces nice looking graphs and figures

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
# This is so figures will display in the notebook without an explicit disp(figure)
%matplotlib inline

### Example of Good Dataset: Southern Ocean Deployment 2 System 1
First, we're going to load, examine, process, and plot an example of a "good" dataset from the Southern Ocean Surface Mooring Deployment 2 METBK 1 (reference designator: GS01SUMO-SBD11-05-METBKA0001)

In [ ]:
# Load the dataset
filepath = '/home/jovyan/wind_comparisons/data/modules/GS01SUMO/R00004/SWND233.nc'
data = xr.open_dataset(filepath)

# Examine the dataset
data.load()

Notice that the dataset has the following "catgories": **Dimensions**, **Coordinates**, **Data variables**, **Indexes**, and **Attributes**. You can read about what each of the categories means in the [xarray documentation](https://docs.xarray.dev/en/stable/user-guide/terminology.html).

In order to reproduce the comparison of scalar wind speed against vector wind speed, we want to find which variables in the dataset represent those data. We can click on the ```Show/Hide Attributes``` button on the far side of each **Data variable** to read the _metadata_ about that variable and learn more about it. We can go ahead and click on _wspd_ variable and see that it is the Scalar averaged wind speed over one minute.

In [ ]:
data

Similarly, we can click on the other data variable's attributes and learn that the other two variables we want are _wnde_, the East Wind vector value, and _wndn_, the North Wind vector value.

There remains one more step before we can get to plotting a comparison, and that is to calculate the vector wind speed average for comparison with the scalar wind speed. If we remember Calculus, we know that:

$$
\|{U}\| = \sqrt{u^{2} + v^{2}}
$$

where $\|{U}\|$ is the magnitude of the wind speed, $u$ is the eastward vector wind speed, and $v$ is the northward vector wind speed. So we can go ahead and calculate that:

In [ ]:
wvec = np.sqrt(data["wnde"]**2 + data["wndn"]**2)
wvec

And we can add the calculate vector wind speed to the dataset to make it easier to access and keep our workspace clean

In [ ]:
# Add the calculated results
data["wvec"] = wvec
# Add some metadata so we can remember what we did
data["wvec"].attrs = {
    'long_name': 'Vector Wind Speed',
    'units': 'm/s',
    'methodology': 'This is the magnitude of the east and north wind vectors'
}

In [ ]:
data

In [ ]:
# Now we can go ahead and plot the data
fig, ax = plt.subplots(figsize=(12, 12))

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

# Plot the one:one line
ax.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax.plot(data["wspd"], data["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Set some limits on the figure
ax.set_xlim((0,25))
ax.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax.set_title(f'Southern Ocean Deployment 2 Sensor S/N {data.attrs["serial_number"]}') # f-strings are a way to substitute a variable into a string and have it print out
ax.set_xlabel('2-axis Sonic Winds (m/s) Scalar')
ax.set_ylabel('2-axis Sonic Winds (m/s) VecAve')
ax.grid()

Great! We now have a comparison plot. But to reproduce what is in the report, we also want to calculate the median value and standard deviation for the vector wind average based on the scalar wind speed.

In order to calculate these values, we are going to make use of the powerful [```groupby```](https://docs.xarray.dev/en/latest/user-guide/groupby.html#) method, which allows you to utilize the **split-apply-combine** strategy. Our strategy is going to be:
1. Group the dataset by the _wspd_ data. We are going to split the dataset into equally-spaced bins based on the scalar wind speed data using the ```groupby_bins``` method in ```xarray```
2. Apply calculations to the grouped data. We want the following values:
    * Mean of _wspd_
    * Median of _wvec_ ($\text{med}(U)$)
    * Mean of _wvec_ ($\bar{U}$)
    * Median standard deviation (msd) of _wvec_. This is calculated as:
$$
\text{msd} = \sqrt{ (\bar{U} - \text{med}(U))^{2} }
$$

3. Plot the data with the binned data

In [ ]:
# First, create the bins we want to group the data into
bins = np.arange(0, 31, 1)
bins

Now group the data into the bins we defined above

In [ ]:
grouped = data.groupby_bins(group='wspd', bins=bins)
grouped

Calculate our statistical values

In [ ]:
# Start with mean of wspd
wspd_avg = grouped.mean()['wspd']

# Next is the median of the vector wind speed
wvec_med = grouped.median()['wvec']

In [ ]:
# To calculate the median standard deviation, we are going to have to write our own function to apply to the grouped data.
def msd(grouped):
    # First, calculate the mean of the vector winds
    wvec_avg = grouped.mean()['wvec']
    
    # Next, calculate the median
    wvec_med = grouped.median()['wvec']
    
    # Use those results to calculate the median standard deviation
    med_std = np.sqrt((wvec_avg - wvec_med)**2)
    
    # Return the results - this is needed to get the result out of the function
    return med_std

In [ ]:
# Calculate the median standard deviation
wvec_msd = msd(grouped)
wvec_msd

Yay! We now have all of the necessary data to reproduce the figure from the report 3305-000207. So lets do it.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Create a one:one line
x = np.arange(0, 31, 1)
y = np.arange(0, 31, 1)

# Plot the one:one line
ax.plot(x, y, color="black", linewidth=2)

# Plot the comparison
ax.plot(data["wspd"], data["wvec"], marker='o', linestyle='', color="red", alpha=0.3) # Alpha controls transparency, 1=solid, 0=transparent

# Plot the bins with error bars
ax.errorbar(x=wspd_avg, y=wvec_med, yerr=2*wvec_msd, fmt='o', linewidth=2, capsize=6)
# Set some limits on the figure
ax.set_xlim((0,25))
ax.set_ylim((0,25))

# Add in title, axis labels, and grid lines
ax.set_title(f'Southern Ocean Deployment 2 Sensor S/N {data.attrs["serial_number"]}') # f-strings are a way to substitute a variable into a string and have it print out
ax.set_xlabel('2-axis Sonic Winds (m/s) Scalar')
ax.set_ylabel('2-axis Sonic Winds (m/s) VecAve')
ax.grid()

Now we want to save the above figure. We'll do this to the same directory where we have the netCDF file. First we'll specify the filepath and create a figure name.

In [ ]:
# First come up with the figure filepath and filename
dirname, basename = os.path.dirname(filepath), os.path.basename(filepath)
# Replace the basename
basename = basename.replace(".nc","")

# Use string concatentation to make the figure filepath
figure_path = dirname + "/" + basename + ".png"
print(figure_path)

Save the figure

In [ ]:
fig.savefig(figure_path, format="png", transparent=False, facecolor="white", edgecolor="black")